In [ ]:
%pip install numpy pandas  matplotlib seaborn
%pip install scikit-learn
!pip install category_encoders
import numpy as np

import pandas as pd

df = pd.read_csv("adult.csv")


from sklearn.preprocessing import StandardScaler,OrdinalEncoder , OneHotEncoder,PolynomialFeatures, PowerTransformer
from sklearn.model_selection import train_test_split 
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer 
from category_encoders import TargetEncoder
from sklearn.model_selection import KFold,cross_val_score
from scipy.stats import zscore
from sklearn.feature_selection import SelectKBest, mutual_info_classif
#from sklearn.feature_selection import SelectKBest,Mutual_info_classif 
from sklearn.linear_model import LogisticRegression
## those are the dependices 


print("hello world")
df.head()## prinitng the first few rows of the data set to see what were working witj
df.isnull().sum()## also finidng the amount of missing values in the data so we can use simple imputer or other data imputation techniques 


x=df.drop("income",axis=1)## we set the x axis to  every column but income
y=df["income"]## we set the y axis to target income
numerical_cols=x.select_dtypes(include=["int64","float64"]).columns## we set numerical columns to every column that has numerical values and then get their names
categorical_cols=x.select_dtypes(include="object").columns## we do the samne here but for categorical columns 
z_scores = np.abs(zscore(x[numerical_cols])) ## we then set z score to be the absoutely value of each z score of every numerical_cols so we can effectively identify outliers 
filterd_df=df[z_scores<3].all(axis=1)## and then we look for every row that has a  standard devation less than 3
x=x[filterd_df]## and then we set this new data frame as x

skewed=  np.abs(x[numerical_cols].skew())## we then find the skew of each column in the numerical_columns , this is to find if its left or right skewed 
skewed_cols = [col for col in numerical_cols if skewed[col] >0.5]## we then check if the skew  is bigger than 0.5 , in which case that means it needs to be transformed 
non_skewed_cols=[col for col in numerical_cols if col not in skewed_cols]## and likewise we make a comprehnsion just incase it is not 
unique_cols = x[categorical_cols].nunique()## we then find out the amount of unique values in each categorical column
small_unique=[col for col in categorical_cols if unique_cols[col]<10]## we sort out the  columns with large uniquye values by seeing if the unique values are less than 10
big_unique=[col for col in categorical_cols if unique_cols[col]>10]## if the unique valuesa are not less than ten then we classify it as a big unique column

skewed_cols_pipeline=Pipeline([## we make a pipeline to transform the skwed columsns , first power transforming it to reduce the skeweness level and then applying standardscaler to make the mean 1 and std =0
    ("power",PowerTransformer(method="yeo-johnson")),
    ("scale",StandardScaler())
])
non_skewed_cols_pipeline=Pipeline([## we simply just scale the non skewed columns
    ("scale",StandardScaler())
])

small_unique_cols_pipeline=Pipeline([## we use one hot encoder on the small unique pieplines , not much risk of creating thousands of  new columns
    ("onehot",OneHotEncoder(handle_unknown="ignore"))
])


big_unique_cols_pipeline = Pipeline([## we use target encoder for the big unique and high caridniltity pipeline , due to the fact it works for massive unique columns much better than most encoding types
    ("target",TargetEncoder())
])
preprocessor = ColumnTransformer([## we use column transformer here to nae
    ("skewed",skewed_cols_pipeline , skewed_cols),
    ("non_skewed_cols",non_skewed_cols_pipeline,non_skewed_cols),
    ("small_unique",small_unique_cols_pipeline,small_unique),
    ("big_unique",big_unique_cols_pipeline,big_unique)
])
final_pipeline =Pipeline([
    ("preprocessor",preprocessor),
    ("feature_selection",SelectKBest(score_func=mutual_info_classif , k = 10)),
    ("polynomial features",PolynomialFeatures(degree=2,include_bias=False,interaction_only=True)),
    ("model",LogisticRegression())
])



kf = KFold(n_splits=5,shuffle=True,random_state=42)## this is done so that to avioid data snooping bias and  data leakage
scores=cross_val_score(final_pipeline,x,y,cv=kf)## this splits the data into 5 folds and each fold gets a chance to be the testing data while the other 4 are the training fold
print(f"the  non skewed columns are {non_skewed_cols}")
print("cross validation scores :{scores}")## this is simply finding the scores of each fold
print(f" the mean scores are {scores.mean()}")## and this is finding the mean , the closer to one the better 

print(f" the skewed_cols are {skewed_cols}")


print ("addional techniques i can use are ")
print ("-"*40)
##.count vectoriser and tdif vectoriser  for things like comments and unique words
##.bayesian smooth encoding to prevent data leakage 
##.custom transformers to fit into pipelines seamlessly
##feature crossing
##.dimensionality reduction with e.g techniques like pca 
##. simple immputer to fill in missing values and also knn imputer and iteratve imputer 
##BIG NOTE - i did not use any of these techniques due to the fact that it wasnt needed on a data set like this



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
hello world


C:\Users\danie\AppData\Local\Temp\ipykernel_22392\3603079639.py:35: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x=x[filterd_df]


ValueError: Found input variables with inconsistent numbers of samples: [0, 32561]